In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/Food Env matching

/content/drive/My Drive/Food Env matching


In [ ]:
import random as rand
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import json
import re
import ast
import os

# Read in the data on food business descriptions

In [ ]:
FSA_data = pd.read_excel("Data/FSA data clean11feb2024.xlsx", sheet_name='clean data')

In [ ]:
# Total number of unique food businesses
len(FSA_data['BusinessName'].unique())

28520

In [ ]:
business_list = FSA_data['BusinessName'].unique().tolist()

In [ ]:
os.path.exists('Data/business_list.txt')

True

In [ ]:
# Save the list of businesses and ensure it is loaded correctly
with open('Data/business_list.txt', 'a') as file:
    for item in business_list_sub:
        file.write(f"{item}\n")

In [ ]:
loaded_list = []
with open('Data/business_list.txt', 'r') as file:
    for line in file:
        # Strip the newline character from each line before appending
        loaded_list.append(line.strip())

# Filter out the matched list those hat had already been excluded.

In [ ]:
len(FSA_data['BusinessName'].unique())

28520

In [ ]:
# List sent by Deksha where some businesses had been manually verified
df = pd.read_excel('Data/food business exclusions/Final list_8000_completed.xlsx', sheet_name = 'Sheet3')

In [ ]:
df['status'].unique()

array(['keep', 'omit', 'Keep', 'Omit'], dtype=object)

In [ ]:
businesses_to_omit = df[df["status"].isin(['omit', "Omit"])]['Names'].unique()
businesses_manually_checked_tokeep =  df[~df["status"].isin(['omit', "Omit"])]['Names'].unique()

In [ ]:
for i in df["Names"]:
  i=str(i)
  if i.startswith("."):
    print(i)

In [ ]:
for i in df['Names'].tolist():
  i =str(i)
  if "Auldhouse".lower() in i.lower():
    print(i)

In [ ]:
df[df["status"].isin(['omit', "Omit"])]['Names']

1                                                     828
25                                     @Home Youth Centre
27                                               1:1 Diet
28      1:1 Diet by Cambridge Weight Plan Samantha Joh...
29      1:1 Diet by Cambridge Weight Plan with Neil an...
                              ...                        
8173                                         YWCA Glasgow
8177                              Zenith Fitness Training
8179                                Zetland Parish Church
8180                          Zetland Parish Church Guild
8186                            Zippy's Gymnastic Academy
Name: Names, Length: 3494, dtype: object

In [ ]:
FSA_data_keep = FSA_data[~FSA_data['BusinessName'].isin(businesses_to_omit)]

In [ ]:
FSA_data_keep['Delete'].unique()

array(['keep', nan, 'care house', 'duplicate', 'old',
       'ask LMJ , keep ASDA café within ASDA store?', 'church',
       ' café at the hospital', 'bank', 'Staff kitchen',
       'catering company', '?', 'Mill', 'football club', 'mill',
       '? Check with LMJ, café within sainbury', 'Keep', 'not FE',
       'staff restaurant', '? Ask LMJ café within store',
       'omit staff canteen, not FE'], dtype=object)

In [ ]:
to_remove = ['care house', 'church', ' café at the hospital', 'bank', 'Staff kitchen',
       'catering company', 'staff restaurant']

In [ ]:
FSA_data_manual_exclusions = FSA_data_keep[~FSA_data_keep["Delete"].isin(to_remove)]

In [ ]:
FSA_data_manual_exclusions.to_excel("Data/food business exclusions/FSA_filtered_plus_manual_excusions.xlsx")

In [ ]:
FSA_data_manual_exclusions[FSA_data_manual_exclusions['Delete']=="church"]

,FHRSID,LocalAuthorityBusinessID,BusinessName,Duplicate,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,RatingDate,LocalAuthorityCode,LocalAuthorityName,geocode.longitude,geocode.latitude,Combined,Combinedloc,Delete


In [ ]:
business_list

In [ ]:
FSA_data_keep["BusinessName"]

In [ ]:
for i in FSA_data_keep['BusinessName'].tolist():
  i = str(i)
  if 'church' in i.lower():
    print(i)

In [ ]:
FSA_data[~FSA_data['BusinessName'].isin(businesses_to_omit)].to_excel('Data/food business exclusions/food_business_data_clean_FSA_filtered.xlsx')

In [ ]:
df[df['status'].isin(['Keep', 'keep'])]['Names'].unique()

array(['88', 1841, 1905, ..., 'ZULUS PERI PERI', 'ZUMO',
       '.Zumo Juice & Smoothie Bar'], dtype=object)

# Prep the prompts

In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken

In [ ]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
MODEL = "gpt-4o"
SYSTEM_ROLE = "You are an expert on food businesses in Scotland, UK"

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="INSERT OWN API KEY")

In [ ]:
########### Some trial prompts that were ultimately not used ######################

# f"""
# For each listed business in {LIST} assess whether it primarily engages in the direct sale of food and/or drink to consumers.
# Consider your knowledge of establishments such as restaurants, cafes, bakeries, snack bars, and similar outlets that serve food and drink directly to patrons based in Scotland.
# Exclude businesses that may produce or distribute food and drink but do not engage in direct sales to consumers in a retail setting.
# Specifically, identify businesses that offer services, goods, or experiences unrelated to the immediate consumption of food and drink, such as catering services, wholesale suppliers, food production not open to public transactions, and other non-retail operations.
# The objective is to filter out businesses that do not offer walk-in or spontaneous food and drink purchasing options to the general public.
# """

# def single_prompt(LIST: list):

#   prompt = f"""Use your extensive domain knowledge of establishments/businesses in Scotland to go through each establishment in the list {LIST} and create a new list that excludes all businesses that sell food and/or drink directly to consumers such as shops, restaurants, newsagents and service stations. If the business name is ambiguous include the business in the output list.
# Only print the new list and nothing else.
#   """
#   return prompt



In [ ]:
def single_prompt_2(LIST: list):

   prompt = f""" Use your extensive domain knowledge of businesses in Scotland to go through the list {LIST} and assess whether it engages in the direct sale of food and/or drink to consumers such as butchers, newsagents, grocery stores, coffee shops, supermarkets, restaurants, cafes, bakeries, butchers, fishmongers and similar outlets.
            If the business DOES NOT meet this criteria, such as a village hall or a catering company, include it in a new list of companies that do not primarily sell food and drink. Only print/output the new list. Do not inlclude any other text in the output.
            """
   return prompt

In [ ]:
def split_list(lst, N):
    k, m = divmod(len(lst), N)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(N)]

In [ ]:
# As we want to extract a python list from the generated string, remove any square bracket that exists in the business name to ensure the list is able to be extracted
bracket_replace = []

for i in business_list:
  if '[' in str(i):
    bracket_replace.append(str(i))

In [ ]:
business_list = [str(s) for s in business_list]

In [ ]:
# replace any square bracket with a normal bracket in the business lists
business_list = [s.replace('[', '(').replace(']', ')') for s in business_list]

In [ ]:
## We also need to ensure that the quotes are always double quotes
def transform_string(input_str):
    transformed_str = re.sub(r"(?<![NL])'(?![sS]\b)", '"', input_str)
    return transformed_str

In [ ]:
# An example
output = transform_string("'SHEDBAL'S RETAIL LTD'")

In [ ]:
print(output)

"SHEDBAL'S RETAIL LTD"


In [1]:
def update_exclusion_list(list_businesses: list, save_path: str):

  sublists = split_list(lst=list_businesses, N=600)

  # In the event of a timeout error, run the matching for a subset of lists
  #sublists = sublists[1:100]

  for count, business_sublist in enumerate(sublists):
    PROMPT = single_prompt_2(LIST = business_sublist)

    response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": SYSTEM_ROLE},
          {"role": "user", "content": f"{PROMPT}"},
      ],
      temperature=0,
    )

    mapping_str_response = response.choices[0].message.content

    try:
      # Find the list object in the string generated by the LLM
      match = re.search(r"\[.*?\]", mapping_str_response, re.DOTALL)
      if match:
        data_str = match.group(0)
        try:
          # Transform the list in the string a python list object
          data_list = ast.literal_eval(data_str)
        except:
          data_str = transform_string(data_str)
          data_list = ast.literal_eval(data_str)

    except:
      print(mapping_str_response)
      break

    if os.path.exists(save_path):
      write_mode = 'a'
    else:
      write_mode = 'w'

    with open(save_path, write_mode) as file:
      for item in data_list:
          file.write(f"{item}\n")

    print(f"finished sublist {count}")

  return

In [ ]:
save_path = 'Data/business_list_exclusions.txt'

In [ ]:
businesses_manually_checked_tokeep.tolist()

['88',
 1841,
 1905,
 21212,
 '.A & A Bros',
 '.A & C Cameron',
 '.A & F Hipwell',
 '.A D Munro',
 '.A D Rattray',
 '.A W HERD AND SONS LTD',
 '.A. Sutherland & Sons',
 '.A.J. Jamieson',
 '.Abbeyfield House',
 '.Abdie Hall',
 '.Abdul Rahman Super Store',
 ".Abdul's Usave",
 '.Aberhill News',
 '.Abertay Services Ltd',
 '.Absorb 3',
 '.ABZ..OLUTE',
 '.Accentric Foods Ltd',
 '.Ach-An-Eas Home',
 '.Achavelgin',
 '.Acorn Centre (Inverurie) Ltd',
 "@Tophski's Ltd.",
 '102 Club',
 '102 Field Squadron',
 '13 Barn St',
 '14/00366/COM',
 '147 Pool & Snooker Centre',
 '147 Snooker Club',
 '1478 Social Club',
 '169 Masonic Social Club',
 '1703 Dunfermline',
 '173 Masonic Club',
 '175 Club Caterer',
 '175 Masonic Club',
 '1802 @ Hutchensons Hall',
 '1871 Lounge',
 '1891 Crieff Golf Club',
 '1hamilton Place',
 '20 Social Club',
 '2nd Chance Inverkeilor Ltd',
 '30 Social Club',
 '32 Social Club',
 '35 Social Club',
 '4 Coo Wynd',
 '443 Club',
 '48 Social Club',
 '612 CLUB',
 '63rd & 1st',
 '65 Club',

In [ ]:
len(businesses_manually_checked_tokeep)

4592

In [ ]:
business_list = FSA_data_manual_exclusions[~FSA_data_manual_exclusions['BusinessName'].isin(businesses_manually_checked_tokeep)]['BusinessName'].unique()

In [ ]:
# remove those that have been already marked for exclusion
for i in business_list:
  i_string = str(i)
  if i_string.startswith("."):
    business_list.remove(i)

In [ ]:
update_exclusion_list(list_businesses=business_list, save_path='Data/business_list_exclusions_prompt2.txt')

In [ ]:
update_exclusion_list(list_businesses=business_list)

finished sublist 0
finished sublist 1
finished sublist 2
finished sublist 3
finished sublist 4
finished sublist 5
finished sublist 6


# Take the finished prompt list and remove the rows from the list

In [ ]:
path = 'Data/business_list_exclusions_prompt2.txt'

business_exclude = []
with open(path, 'r') as file:
    for line in file:
        # Strip the newline character from each line before appending
        business_exclude.append(line.strip())

In [ ]:
len(business_exclude)

8364

In [ ]:
# Check for items that were halluciantde by the model
hallucinated_items = []

for i in business_exclude:
  if i not in business_list:
    hallucinated_items.append(i)

In [ ]:
keywords = ['Woodend Bowling Club']

for i in business_list:
  i = str(i)
  if all(keyword.lower() in i.lower() for keyword in keywords):
    print(i)

WOODEND BOWLING CLUB


In [ ]:
"""
Example hallucination:

Generated                             | Original
-------------------------------------------------------------------------------------------
'A And M Catering - Street Trader'    |  A and M Catering - Street Trader   --> changing case
'Delta Facility'                      |  DELTA Facility
 "ELIOR @ BABCOCK [Terminal 2]"       | ELIOR @ BABCOCK (Terminal 2) --> changing brackets
 'Glasgowean Health Shop'             | "Glaswegian Health Shop"  --> mispelling
'Hill Of Banchory School'             |   "Hill of Banchory School"

"""

In [ ]:
34/8364

0.0040650406504065045

In [ ]:
hallucinated_items

['88',
 '828',
 '1841',
 '1905',
 '21212',
 'A And M Catering - Street Trader',
 'Delta Facility',
 'ELIOR @ BABCOCK (Terminal 2)',
 'Glasgowean Health Shop',
 'GLOW With Gayle',
 'Govan Community Fire Station',
 'GRANITE CITY  (Landside)',
 'HALFORDS (RETAIL SHOP ONLY)',
 'HAZLEHEAD GOLF CLUB (Bar only)',
 'Hill Of Banchory School',
 'Hobbycraft',
 'J and JG Dickson And Son Ltd',
 'Linkes (SCIO)',
 'MacKays Ltd. Retail Shop',
 'Maitland Bowling Club',
 'Motor Fuel Group Ltd (Grantown))',
 'NATURES WAY (CHINESE HERBAL MEDICINE)',
 'Option One Events',
 'Piazza News',
 'Pollok House',
 'Reliance @ ABDN SHERIFF COURT (CELL AREA',
 'SCAMPS (AFTER SCHOOL CLUB)',
 'SODEXO @ SCOTTISH GOVERNMENT (MARINE LAB)',
 'SODEXO @ SHELL (WOODBANK)',
 'ST MARYS (SHARED) CHURCH',
 'SUPERDRUG (UNIT 15)',
 'THE 1224 CLUB (MASONIC LODGE)',
 'The Balloon And Chocolate Company . . .',
 'The KEY',
 'The Tunnels',
 'W H SMITH (AIRSIDE)',
 'W H SMITH (LANDSIDE)',
 'W H SMITH (LANDSIDE) (ARRIVALS)',
 'Woodend Bow

In [ ]:
# newsagents/service stations and a number of other businesses were included in the list of non-food businesses: manually reintroduce to list of businesses to include.

keyword_exclusions = []

In [ ]:
keywords = ['Auldhouse']
count = 0
for i in business_exclude:
  if all(keyword.lower() in i.lower() for keyword in keywords):
    print(i)

In [ ]:
keywords = ['service', 'station']
count = 0
for i in business_exclude:
  if all(keyword.lower() in i.lower() for keyword in keywords):
    count += 1

print(count)

62


In [ ]:
keywords = ['service', 'station']

for i in business_exclude:
  if all(keyword.lower() in i.lower() for keyword in keywords):
    keyword_exclusions.append(i)

In [ ]:
keyword_exclusions += ['Portknockie Fish & Chip Shop']

In [ ]:
keyword_exclusions

In [ ]:
keyword_exclusions = ['A Warnock Butchers',
 'A Weir - Butcher',
 'A. Warnock Butcher',
 'Alan Elliot Butchers Ltd',
 'Alan Scobbie Butchers',
 'Brian Henderson Family Butchers',
 'Brunton Butchers',
 'Bryan Beveridge Quality Butcher',
 'Burnbank Butchers Ltd',
 'El Madinah Butchers',
 'Events By Bannerman Butcher',
 'Graeme Barber Butcher Ltd',
 'Portree Butchers',
 'Street Trader - Margaret Butcher',
 'The Cross Butchers Ltd',
 'The Cumbrae Butcher',
 'The Spanish Butcher',
 'W J Curley (Butchers)',
 'W Lambie Butcher',
 'A & J Newsagent',
 'ALI AND SONS NEWSAGENT',
 'Buchanan Newsagents Ltd',
 'Craigend Newsagent',
 "Craig's Newsagent",
 'Crossroads Newsagents',
 'Curtis Newsagents',
 'Dils Newsagents',
 'Duff Newsagents',
 'Dunipace Newsagent',
 'Eastend Newsagents Shop Local',
 'Economy Store Newsagents',
 "Edgar's Newsagent",
 'Elie Newsagents',
 'First Stop Newsagent',
 'Fisherrow Newsagents',
 'Freddies Newsagent',
 'Gilroy Newsagents',
 'Hassan Newsagents (Partnership)',
 'HENDERSON NEWSAGENTS',
 "Henderson's Newsagent",
 'HUTCHISON NEWSAGENT',
 'I & E Wales Newsagents',
 'Iain Hamilton Newsagents',
 'Jassal Newsagent',
 'Johns Newsagent',
 'Kanco Newsagents',
 'Kanco Newsagents And Grocers',
 'KEIR STREET NEWSAGENT',
 "Khan's Newsagent And Post Office",
 'Kings Road Newsagent',
 'Laidlaws Newsagents Ltd',
 'Lennox Newsagents',
 "Livingston's Newsagent",
 'M & D CALEDONIAN NEWSAGENTS',
 'Madina Newsagents',
 'Maxwellton Newsagents',
 'McGuinness Newsagents',
 'Moss Road Newsagents',
 'Mosspark Newsagent /  U Save',
 'My Newsagents',
 'Nannys Newsagents',
 'Neville Newsagents',
 'News First Newsagents',
 'Newsagent Store',
 'Newsagents',
 'NEWTON NEWSAGENT',
 'R and R Parry (newsagents)',
 'Ritchies Newsagent',
 'Rite Price Newsagent',
 "Sanjay's Newsagents",
 'SEAFIELD NEWSAGENT',
 'Sialvis Newsagent',
 'Sighthill Newsagent',
 'SIMPSONS NEWSAGENTS',
 'Singhs Newsagent',
 'Station Newsagent',
 'Sugar Rush Newsagents',
 'Summerston Newsagents and General Store',
 'Sun Newsagents',
 'T S Newsagents',
 'Tara Newsagent',
 'Tariq Newsagents',
 'The Late Shop/ Cessnock Newsagent',
 'The Squarie Newsagent Ltd',
 'Toll Newsagents',
 'Tollcross Newsagent',
 'Tormusk Newsagent',
 'Union Road Newsagent',
 "Vaneesa's Newsagent",
 'Village Newsagent',
 'W And T Gough Newsagents',
 'W G Stars Newsagents',
 'Walkers Newsagents',
 'Watch Shop And First Stop Newsagent',
 'Westwoods Newsagents',
 "Wylie's Newsagents",
 'Yalda Newsagents',
 'Zaveri Newsagent',
 'A M Service Station',
 'Abbotsinch Service Station',
 'Alexandria Service Station',
 'Bothwell Service Station',
 'Bothwellpark Service Station Ltd',
 'BP Service Station',
 'BP Seven Sisters Service Station',
 'Braemar Service Station',
 'Bridgend Service Station',
 'Broomfield Service Station (Londis)',
 'Buccaneer Service Station',
 'Buchandyke Service Station',
 'Coylton Service Station',
 'Crossburn Service Station',
 'Dalmarnock Service Station (ESSO)',
 'Dalwhinnie Service Station',
 'Dumbarton Service Station',
 'Earnock Service Station',
 'FIVE MILE SERVICE STATION',
 'Fullarton Service Station',
 'Garrowhill Service Station',
 'Gulf Coatbridge Service Station',
 'Hamilton Service Station',
 'Hogganfield Loch Service Station',
 'Ibrox Service Station',
 'Kildean Service Station',
 'Kilwinning Service Station',
 'Kirkmuirhill Service Station',
 'Lennox Service Station',
 'Loans Service Station',
 'Lomond Service Station Ltd',
 'Masonhill Service Station',
 'Maxwell Arms Service Station',
 'Mayfield Service Station',
 'Menstrie Service Station',
 'MFG Calder Service  Station (Shop)',
 'MFG Pitlochry Service Station',
 'Mosstodloch Service Station',
 'Motherwell Service Station',
 'Motor Fuel Group Ltd (Kessock Service Station)',
 'Mountfleurie Service Station',
 'Paisley Mill Service Station',
 'Park Orchardbank Service Station',
 'Penny Petroleum UK Ltd (ESSO Ben Nevis Service Station)',
 'Queenslie Service Station',
 'Rutherglen Service Station',
 'Shell Westermains Service Station',
 'Sound Service Station Ltd',
 'South Park Service Station',
 'SRS Service Station',
 'Switchback Service Station',
 'The Rigg Service Station/Belhaven Bikes',
 'Toll Service Station',
 'Townfoot Service Station',
 'Turriff Service Station',
 'Uddingston Service Station',
 'Viewpark Service Station',
 'Watermill Service Station',
 'Wemyssfield Service Station',
 'Western Highway Service Station',
 'Wigtown Road Service Station',
 'Wishaw Service Station (SHOP)',
 'Portknockie Fish & Chip Shop']

In [ ]:
business_exclude_filtered = [i for i in business_exclude if i not in keyword_exclusions]

In [ ]:
len(business_exclude_filtered)

8203

In [ ]:
with open('Data/business_list_exclusions_GPT_string.txt', 'w') as file:
      for item in business_exclude_filtered:
          file.write(f"{item}\n")